In [1]:
import numpy as np
import os
import torch
import pandas as pd
import csv


In [56]:
info = pd.read_csv('/home/duyongkang/aminer/aminer.paper.info.csv')
ppr_list = info['paper_id'].values.tolist()             # 论文列表
text_list = info['text'].values.tolist()                # 文本列表
cit_list = info['citations'].values.tolist()            #引用列表
confer_list = info['conference'].values.tolist()        #会议列表

# 论文的引用
cit_dic = {}      # key为论文编号，value为引用论文编号
for i in range(len(ppr_list)):
    ppr = 'p' + str(ppr_list[i])
    cit_dic[ppr] = []
    lenth = len(cit_list[i])
    if lenth == 1:
        continue
    elif lenth > 1 and lenth < 8:
        cit = 'p' + str(cit_list[i])
        cit_dic[ppr].append(cit)
    elif lenth > 8:
        cit = cit_list[i].split(';')
        cit = ['p'+num for num in cit]
        cit_dic[ppr].extend(cit)
        
def Cit_Paper( paper ):
    return cit_dic[paper]

# 同一个会议的论文
pc_dic = dict(zip(ppr_list,confer_list))              #key为论文，value为会议
confer_dic = {}
for i in range(len(ppr_list)):
    confer = confer_list[i]
    ppr = 'p' + str(ppr_list[i])
    if confer not in confer_dic.keys():
        confer_dic[confer] = []
    confer_dic[confer].append(ppr)
import copy   
def Same_Confer_Paper( paper ):
    confer = pc_dic[int(paper[1:])]
    temp = copy.deepcopy(confer_dic[confer])
    temp.remove(paper)
    return temp

In [165]:
# p904181
list1 = Same_Confer_Paper('p904181')
list2 = Cit_Paper('p904181')
list1.extend(list2)
list1 = list(set(list1))

In [29]:
# 通过a-p-a-p找到的论文
ap_info = pd.read_csv('/home/duyongkang/aminer/aminer.author.paper.link.csv')
ppr_list = ap_info['paper_id'].values.tolist()
auth_list = ap_info['author_id'].values.tolist()

# 统计每篇文章的作者，key为论文标号，value为作者列表
pa_dic = {}
for i in range(len(ppr_list)):
    p = 'p' + str(ppr_list[i])
    a = 'a' + str(auth_list[i])
    try:
        pa_dic[p].append(a)
    except KeyError:
        pa_dic[p] = []
        pa_dic[p].append(a)

# 统计每个作者的文章，key为作者编号，value为论文列表
ap_dic = {}
for i in range(len(auth_list)):
    p = 'p' + str(ppr_list[i])
    a = 'a' + str(auth_list[i])
    try:
        ap_dic[a].append(p)
    except KeyError:
        ap_dic[a] = []
        ap_dic[a].append(p)

In [34]:
# 在pa字典里找到该论文的其他作者，再回到ap字典中找这些作者写作的论文
paper = 'p1872287'
authors = pa_dic[paper]
authors

['a867104', 'a99940']

In [36]:
authors.remove('a99940')
authors

['a867104']

In [37]:
other_papers = ap_dic[authors[0]]
other_papers

['p1227651', 'p1872287']